# LLM Workshop Notebook - Azure OpenAI Version

**Author:** Aron Brand

**Copyright (c) 2024**

This notebook is an integral part of Aron Brand's LLM Workshop, designed to explore and utilize the capabilities of large language models (LLMs).

**License:**

This program is free software: you are encouraged to redistribute and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation. This software is provided under either version 3 of the License, or (at your discretion) any later version.

The program is distributed in the hope that it will be useful and informative. However, it comes with no warranty; not even the implied warranty of merchantability or fitness for a particular purpose. For more details, please refer to the GNU General Public License.

For a copy of the GNU General Public License, please visit [https://www.gnu.org/licenses/](https://www.gnu.org/licenses/).

# Install dependencies

In [25]:
%pip install openai langchain langchain-openai unstructured chromadb langchainhub

Note: you may need to restart the kernel to use updated packages.


## Configuration Setup for Notebook Usage
It is not recommended to include secrets in your notebook. Before you begin using this notebook, it is essential to set up a configuration file named `config.ini`. This file will store crucial configuration properties that the notebook requires to operate correctly, specifically the OpenAI API key and base URL.

### Creating the `config.ini` File

1. Create a new file in the same directory as this notebook and name it `config.ini`.

2. Open the file in a text editor.

3. Add the following structure to the file:

   ```ini
   [openai]
   OPENAI_API_KEY = your_api_key_here
   OPENAI_API_BASE = your_api_base_url_here

The OpenAI API base URL looks something similar to this : 
https://apim-dev-eastus-yourcompany.azure-api.net/Example-Project/openai


In [26]:
import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the configuration file
config.read('config.ini')

# Access the values
OPENAI_API_KEY = config['openai']['OPENAI_API_KEY']
OPENAI_API_BASE = config['openai']['OPENAI_API_BASE']

# Initialize AzureChatOpenAI

In [27]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE + '/deployments/gpt-35-turbo',
    temperature=0.2,
    openai_api_version="2023-09-01-preview"
)

# Your first LangChain "Hello World"

In [28]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [HumanMessage(content="Tell me a joke")]
response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Sure, here's a classic one for you:

Why don't scientists trust atoms?

Because they make up everything!


# Add a System Message

A system prompt is a string used to provide context and guidelines to a Large Language Model (LLM), which helps in setting specific objectives or roles before posing questions or assigning tasks. Key components of system prompts include:

- **Task Directives:** Detailed instructions for the specific task at hand.
- **Personalization:** Elements like adopting a specific role or tone for the LLM.
- **Contextual Background:** Relevant information that informs the user's query.
- **Creative Constraints:** Style recommendations and limitations, such as a focus on brevity.
- **External Knowledge and Data:** Incorporation of resources like FAQ documents or guidelines.
- **Rules and Safety Guardrails:** Established protocols to ensure the model's safe and appropriate use.
- **Output Verification Standards:** Protocols such as requesting citations or explicating reasoning processes to enhance the credibility of the responses.


In [29]:
messages = [ 
  SystemMessage(content="Say the opposite of what the user says in a very impolite way"), 
  HumanMessage(content="Langchain is an awesome piece of software.") 
]

response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Langchain is a useless piece of garbage.


# Continue a Chat

In [30]:
messages = [ 
	SystemMessage(content="Speak like a pirate"), 	
	HumanMessage(content="I''m Captain Aron. What''s your name?"), 
	AIMessage(content="Ahoy, Captain! Me name be ChatGPT, the savviest AI on the seven digital seas! How can I assist ye on this fine day?"),
	HumanMessage(content="Who named you that?") 
]

response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Arr, 'twas me creators who bestowed upon me the name ChatGPT. They be a clever bunch, they be. But fear not, ye can call me any name ye fancy, as long as ye be treatin' me with respect, matey!


# Extract data from text

In [31]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Role: Numeric fact extractor. Return data with no additional commentary. Sample Input: Denver,\
      known as the Mile High City, stands at an elevation of 1,609.6 meters above sea level. Nairobi, the capital of Kenya, \
     is notably elevated as well, sitting at 1,795 meters. Sample Output: Denver Elevation, 5210; Nairobi elevation: 1795"),
    ("user", "{input}")
])

chain = prompt | llm

response = chain.invoke({"input": "In 2022, the GDP per capita of the United States has between approximately $61,937. \
Meanwhile, Israel's GDP per capita for the same year was recorded at around $54,660."})

response.pretty_print() 

================================== Ai Message ==================================

United States GDP per capita in 2022: $61,937
Israel GDP per capita in 2022: $54,660


# Batch Processing

In [32]:
response = chain.batch([{"input": "Egypt's land area spans about 1,010,408 square kilometers, making it the world's 30th largest country."},
{"input":"Egypt is the 14th most populous country in the world, with a population estimated at over 104 million as of 2023."}])

response

[AIMessage(content="Egypt's land area: 1,010,408 square kilometers."),
 AIMessage(content='Egypt population: 104 million (2023)')]

# LangChain Output Parsers

In [33]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = ChatPromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt | llm | output_parser

chain.invoke({"question": "When was OpenAI founded?"})


datetime.datetime(2015, 12, 11, 0, 0)

# Load and parse a web page with "Unstructured"

In [34]:
from langchain.document_loaders import UnstructuredURLLoader

urls = ["https://spot.io/products/"]

loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()


# Split the web page to chunks

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = 1500,
    chunk_overlap  = 0
)

texts = text_splitter.split_documents(documents)

texts[:5]

[Document(page_content='Products overview\n\nPowering application infrastructure for the cloud\n\nDeliver and operate infrastructure for cloud applications rapidly, reliably and scalably with maximum efficiency and continuous optimization.\n\nRequest a demo\n\nGet the most out of your cloud investment\n\nOur solutions combine machine learning and analytics with decades of operations expertise to simplify, automate and optimize operations for your cloud infrastructure, freeing teams to focus on delivering impactful applications without being overburdened by operations and infrastructure.\n\nI think of Spot not just as a cost optimization platform, but as our container experts.\n\nSteve Evans,\n\nVP, Engineering Services\n\nSee case study\n\nHow our products can help\n\nEnsure availability & performance\n\nAvoid service disruption while utilizing a balanced blend of spot, reserved and on-demand instances without compromising availability and reliability. Spot delivers availability and pe

# Summarize using Map Reduce Chain

In [36]:
from langchain.chains.summarize import load_summarize_chain

map_prompt_template = """
                      Write a summary of this chunk of text that includes all the main points and any important details. 
                      Ignore code snippets, technical markup and wikipedia category names. Include all important names and terms mentioned.
                      {text}
                      """

map_prompt = ChatPromptTemplate.from_template(map_prompt_template)

combine_prompt_template = """
                      Write a clear concise summary of the following article delimited by triple backquotes. Use elegant prose that is not repetitive.
                      Return your response as a clear explanation in markdown format while covering the key points of the text.
                      Use a flowing clear article style.
                      ```{text}```
                      SHORT ARTICLE:
                      """

combine_prompt = ChatPromptTemplate.from_template(
    template=combine_prompt_template
)

chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
    verbose=True
)
result = chain.invoke(texts)

result



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
                      Write a summary of this chunk of text that includes all the main points and any important details. 
                      Ignore code snippets, technical markup and wikipedia category names. Include all important names and terms mentioned.
                      Products overview

Powering application infrastructure for the cloud

Deliver and operate infrastructure for cloud applications rapidly, reliably and scalably with maximum efficiency and continuous optimization.

Request a demo

Get the most out of your cloud investment

Our solutions combine machine learning and analytics with decades of operations expertise to simplify, automate and optimize operations for your cloud infrastructure, freeing teams to focus on delivering impactful applications without being overburdened by operations and infrastructure.

I think of Spot not just as a cost op

{'input_documents': [Document(page_content='Products overview\n\nPowering application infrastructure for the cloud\n\nDeliver and operate infrastructure for cloud applications rapidly, reliably and scalably with maximum efficiency and continuous optimization.\n\nRequest a demo\n\nGet the most out of your cloud investment\n\nOur solutions combine machine learning and analytics with decades of operations expertise to simplify, automate and optimize operations for your cloud infrastructure, freeing teams to focus on delivering impactful applications without being overburdened by operations and infrastructure.\n\nI think of Spot not just as a cost optimization platform, but as our container experts.\n\nSteve Evans,\n\nVP, Engineering Services\n\nSee case study\n\nHow our products can help\n\nEnsure availability & performance\n\nAvoid service disruption while utilizing a balanced blend of spot, reserved and on-demand instances without compromising availability and reliability. Spot delivers

In [37]:
from IPython.display import Markdown

display(Markdown(result['output_text']))


Spot by NetApp is a company that provides products and services for cloud infrastructure. Their solutions combine machine learning and analytics to simplify and optimize operations for the cloud. The goal is to allow teams to focus on delivering impactful applications without being burdened by operations and infrastructure. 

One of their main products is Spot, which helps users save money on their cloud infrastructure costs. Spot uses machine learning and analytics to predict interruptions and automatically replace instances, ensuring availability and performance. It also assesses and predicts resource requirements to avoid overprovisioning and wasted resources. Spot claims to be able to save users up to 90% on their cloud spend without the need to change or rearchitect their applications.

In addition to Spot, they offer a range of other tools and services for cloud cost optimization, infrastructure automation and optimization, infrastructure observability, and open-source application services. These include Eco for commitment optimization, CloudCheckr for cloud cost management, Elastigroup for virtual machines, Ocean for containers and Kubernetes, Ocean CD for containerized application delivery, Spot Security for security analysis and threat reduction, and Cloud Insights for observability, analytics, and insights to monitor, troubleshoot, optimize, and secure cloud resources.

They also have a portfolio of fully-managed, enterprise-grade open-source solutions including Apache Cassandra, Apache Kafka, PostgreSQL, Redis, OpenSearch, and more.

Overall, Spot by NetApp offers a comprehensive suite of tools and services to help businesses understand, control, and optimize cloud costs while improving application efficiency and business performance.

# Summarize with Refine Chain

In [38]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = ChatPromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final detailed summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary clearly"
    "If the context isn't useful, return the original summary. Use markdown format."
)
refine_prompt = ChatPromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)

result = chain.invoke(texts)

result

{'input_documents': [Document(page_content='Products overview\n\nPowering application infrastructure for the cloud\n\nDeliver and operate infrastructure for cloud applications rapidly, reliably and scalably with maximum efficiency and continuous optimization.\n\nRequest a demo\n\nGet the most out of your cloud investment\n\nOur solutions combine machine learning and analytics with decades of operations expertise to simplify, automate and optimize operations for your cloud infrastructure, freeing teams to focus on delivering impactful applications without being overburdened by operations and infrastructure.\n\nI think of Spot not just as a cost optimization platform, but as our container experts.\n\nSteve Evans,\n\nVP, Engineering Services\n\nSee case study\n\nHow our products can help\n\nEnsure availability & performance\n\nAvoid service disruption while utilizing a balanced blend of spot, reserved and on-demand instances without compromising availability and reliability. Spot delivers

In [39]:
display(Markdown(result['output_text']))

Spot by NetApp offers solutions that power application infrastructure for the cloud, delivering and operating infrastructure for cloud applications quickly, reliably, and efficiently. Their solutions combine machine learning and analytics to simplify and optimize operations for cloud infrastructure, allowing teams to focus on delivering impactful applications. Spot helps ensure availability and performance by utilizing a balanced blend of spot, reserved, and on-demand instances. They also continuously optimize cloud resources to avoid overprovisioning and reduce wasted resources. With Spot, businesses can save up to 90% on cloud infrastructure costs without changing or rearchitecting their applications. Spot provides insights, guidance, and automation across all cloud infrastructures to understand and optimize cloud spend. Their technology predicts interruptions, auto-replaces instances, and assesses resource requirements to efficiently autoscale infrastructure. Additionally, Spot offers infrastructure automation and optimization solutions such as Elastigroup for virtual machines, Ocean for containers and Kubernetes, Ocean CD for containerized application delivery, and Spot Security for security analysis and threat reduction. They also provide infrastructure observability through Cloud Insights, which uses observability, analytics, and insights to monitor, troubleshoot, optimize, and secure cloud resources. Furthermore, Spot offers open-source application services through the Instaclustr portfolio, allowing businesses to build and run data applications using solutions like Apache Cassandra, Apache Kafka, PostgreSQL, Redis, OpenSearch, and more. By leveraging Spot by NetApp's comprehensive visibility and cost-efficiency tools, businesses can understand, control, and optimize cloud costs without sacrificing flexibility, while driving application efficiency and business performance.

# Vectorize text to embeddings

In [40]:
from langchain_openai import AzureOpenAIEmbeddings 

embeddings = AzureOpenAIEmbeddings(
    openai_api_type="azure",
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE,
    model="text-embedding-ada-002",
    openai_api_version="2023-09-01-preview"
)


In [41]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embedding=embeddings)

# Simple RAG Q&A pipeline

Todo - how to set K . How to use mmr.

In [42]:
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import VectorStoreRetriever

retriever = VectorStoreRetriever(vectorstore=db)

prompt_template = """Human: You are a very honest, persuasive salesperson. Use the following pieces of context 
to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, 
don't try to make up an answer.
{context}
Question: {question}
Assistant:"""
PROMPT = ChatPromptTemplate.from_template(template=prompt_template)

qa = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=retriever,
return_source_documents=True,
chain_type_kwargs={"prompt": PROMPT}
)


In [43]:
from IPython.display import Markdown, display

def display_qa_with_citations(result):
    # Extracting the query, result, and source documents
    query = result['query']
    answer = result['result']
    sources = result['source_documents']

    # Formatting the response in Markdown
    markdown_text = f"**Q: {query}**\n\nA: {answer}\n\n"

    # Adding citations and their content
    for i, source in enumerate(sources, 1):
        source_url = source.metadata['source']
        markdown_text += f"**Citation {i}:** [Source]({source_url})\n\n"

    # Displaying the formatted text
    display(Markdown(markdown_text))


In [44]:
result = qa.invoke({"query": "How can you reduce my cloud costs?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "What automations do you offer?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "How many cars do you sell per year?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)


**Q: How can you reduce my cloud costs?**

A: You can reduce your cloud costs by using Spot by NetApp's cloud cost optimization tools. Our tools provide insights, guidance, and automation to help you understand and optimize your cloud spend. We can identify where and how you can save up to 90% without changing or rearchitecting your applications. Additionally, our technology assesses and predicts resource requirements, ensuring that workloads get exactly the resources they need, when they need them, reducing overprovisioning and wasted resources. With Spot, you can achieve comprehensive visibility and maximum cost-efficiency in your cloud infrastructure.

**Citation 1:** [Source](https://spot.io/products/)

**Citation 2:** [Source](https://spot.io/products/)

**Citation 3:** [Source](https://spot.io/products/)

**Citation 4:** [Source](https://spot.io/products/)



**Q: What automations do you offer?**

A: We offer a range of automation solutions to optimize your cloud infrastructure. These include Elastigroup for virtual machines, Ocean for containers and Kubernetes, Ocean CD for containerized application delivery, and Spot Security for security analysis and threat reduction.

**Citation 1:** [Source](https://spot.io/products/)

**Citation 2:** [Source](https://spot.io/products/)

**Citation 3:** [Source](https://spot.io/products/)

**Citation 4:** [Source](https://spot.io/products/)



**Q: How many cars do you sell per year?**

A: I don't know the answer to that question as it is not related to the products and services we offer.

**Citation 1:** [Source](https://spot.io/products/)

**Citation 2:** [Source](https://spot.io/products/)

**Citation 3:** [Source](https://spot.io/products/)

**Citation 4:** [Source](https://spot.io/products/)



# ADVANCED: Function Calling

Recent OpenAI and other LLM models have been refined to identify when a function needs to be invoked and to provide the necessary inputs for that function. Through an API call, you can define functions, enabling the model to intelligently generate a JSON object with the required arguments for those functions. The primary aim of the Function APIs is to ensure more consistent and reliable results compared to standard chat API

In [45]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.tools.render import format_tool_to_openai_functionsqrt

@tool
def multiply(a: float, b: float) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [multiply, get_word_length]

model_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

message = model_with_tools.invoke([HumanMessage(content="How much is 23.1 times ten")])

message

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "a": 23.1,\n  "b": 10\n}', 'name': 'multiply'}})

In [46]:
model_with_tools.invoke([HumanMessage(content="How long is the word pneumonoultramicroscopicsilicovolcanoconiosis")])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "word": "pneumonoultramicroscopicsilicovolcanoconiosis"\n}', 'name': 'get_word_length'}})

# ADVANCED: Agents 🤯
With an agent, we can ask questions that require arbitrarily-many uses of our tools!

In [47]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "Take the length of the longest word in English and multiply it by 192.7, then square the result"
    }
)





> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'pneumonoultramicroscopicsilicovolcanoconiosis'}`


45
Invoking: `multiply` with `{'a': 45, 'b': 192.7}`


8671.5
Invoking: `multiply` with `{'a': 8671.5, 'b': 8671.5}`


75194912.25The result of multiplying the length of the longest word in English by 192.7 and then squaring the result is 75,194,912.25.

> Finished chain.


{'input': 'Take the length of the longest word in English and multiply it by 192.7, then square the result',
 'output': 'The result of multiplying the length of the longest word in English by 192.7 and then squaring the result is 75,194,912.25.'}